In [1]:
#Import Libraries and Capture Camera
import cv2         # OpenCV - image processing
import numpy as np # NumPy - for array operations
import math        # Math - for angle calculations (used later in finger count)

# Starting the webcam and displaying the live feed
cap = cv2.VideoCapture(0)

# Dummy function for trackbars
def nothing(x):
    pass

# Creating Trackbar window
cv2.namedWindow("Color Adjustments",cv2.WINDOW_NORMAL)
cv2.resizeWindow("Color Adjustments", 300,300)
# Create a trackbar to adjust thresholding
cv2.createTrackbar("Thresh", "Color Adjustments", 0, 255, nothing) 

# Creating trackbars for Hue, Saturation, and Value ranges - (COLOR Detection Trackbars)
cv2.createTrackbar("Lower_H", "Color Adjustments", 0, 255, nothing)
cv2.createTrackbar("Lower_S", "Color Adjustments", 0, 255, nothing)
cv2.createTrackbar("Lower_V", "Color Adjustments", 0, 255, nothing)
cv2.createTrackbar("Upper_H", "Color Adjustments", 255, 255, nothing)
cv2.createTrackbar("Upper_S", "Color Adjustments", 255, 255, nothing)
cv2.createTrackbar("Upper_V", "Color Adjustments", 255, 255, nothing)

# Start capturing frames - INSIDE while loop
while True:
    success, frame = cap.read()     # Webcam se ek frame read karo
    if not success:
        break

    frame = cv2.flip(frame, 2)               # # Flip the frame horizontally (like a mirror view)
    frame = cv2.resize(frame, (600, 500))     # Resize for consistency

    # Get hand data from Rectangle(like sub window) and Crop it for processing purpose
    cv2.rectangle(frame, (0, 1), (300, 500), (255, 0, 0), 0) # Draw a rectangle as hand detection area
    crop_image = frame[1:500, 0:300]                         # Crop only the region inside the rectangle for processing ( sirf rectangle wala part (ROI) cut kiya jata hai )

   # Convert cropped image from BGR to HSV
    hsv = cv2.cvtColor(crop_image, cv2.COLOR_BGR2HSV)

   # Detecting Hand (from Trackbars)
    l_h = cv2.getTrackbarPos("Lower_H", "Color Adjustments")
    l_s = cv2.getTrackbarPos("Lower_S", "Color Adjustments")
    l_v = cv2.getTrackbarPos("Lower_V", "Color Adjustments")
    u_h = cv2.getTrackbarPos("Upper_H", "Color Adjustments")
    u_s = cv2.getTrackbarPos("Upper_S", "Color Adjustments")
    u_v = cv2.getTrackbarPos("Upper_V", "Color Adjustments")

    #Create Lower and Upper HSV Ranges
    lower_bound = np.array([l_h, l_s, l_v])  # Lower HSV bound from trackbars
    upper_bound = np.array([u_h, u_s, u_v])  # Upper HSV bound from trackbars

    #Creating mask
    mask=cv2.inRange(hsv, lower_bound, upper_bound)  # Create binary mask using HSV range
    # Filter mask with original cropped image
    filtr = cv2.bitwise_and(crop_image, crop_image, mask=mask) #filters the hand from the original image using the mask

    # Invert the binary mask to make hand white and background black (useful for contour detection)
    mask1 = cv2.bitwise_not(mask)  

    # Get the current threshold value from the trackbar (adjustable by user)
    m_g = cv2.getTrackbarPos("Thresh", "Color Adjustments")  # Get the current threshold value from the trackbar (adjustable by user)

    # Apply binary threshold to the inverted mask to get a clean black & white image
    ret, thresh = cv2.threshold(mask1, m_g, 255, cv2.THRESH_BINARY)  

    # Dilate the thresholded image to enhance and thicken the hand region (fills small gaps)
    dilate = cv2.dilate(thresh, (3, 3), iterations=6)  
   
    #Find contours (outlines) in the thresholded image
    cnts, hier = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    try:

        #Find contour with maximum area
        cm = max(cnts, key=lambda x: cv2.contourArea(x))
        # Approximate the contour to reduce the number of points
        epsilon = 0.0005 * cv2.arcLength(cm, True) #Calculate approximation accuracy (epsilon)
        data = cv2.approxPolyDP(cm, epsilon, True) # Simplify the contour shape to reduce points while preserving overall shape
        # Find the convex hull of the largest contour
        hull = cv2.convexHull(cm) #(for outer shape of hand)
        
        # Draw the actual contour in purple-ish color
        #cv2.drawContours(image, contours, contourIndex, color, thickness)
        cv2.drawContours(crop_image, [cm], -1, (50, 50, 150), 2) # Draw the contour on the cropped image (purple)

        # Draw the convex hull in green
        cv2.drawContours(crop_image, [hull], -1, (0, 255, 0), 2) # Draw the convex hull on the cropped image (green)
        
        
        # Find convexity defects
        hull = cv2.convexHull(cm, returnPoints=False)
        defects = cv2.convexityDefects(cm, hull)
        count_defects = 0
        #print("Area==",cv2.contourArea(hull) - cv2.contourArea(cm))
        for i in range(defects.shape[0]):
            s,e,f,d = defects[i,0]
           
            start = tuple(cm[s][0])
            end = tuple(cm[e][0])
            far = tuple(cm[f][0])
            
            #Cosin Rule
            a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
            b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
            c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
            angle = (math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c)) * 180) / 3.14
            #print(angle)
            # if angle > 50 draw a circle at the far point
            if angle <= 50:
                count_defects += 1
                cv2.circle(crop_image,far,5,[255,255,255],-1)
        
        print("count==",count_defects)
        
        
        # Print number of fingers
        if count_defects == 0:
            
            cv2.putText(frame, "1", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255),2)
        elif count_defects == 1:
            cv2.putText(frame, "2", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        elif count_defects == 2:
            
            cv2.putText(frame, "3", (5, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        elif count_defects == 3:
            
            cv2.putText(frame, "4", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        elif count_defects == 4:
            
            cv2.putText(frame, "5", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        else:
            pass
           
    except:
        pass
        
    cv2.imshow("Thresh", thresh)
    #cv2.imshow("mask==",mask)
    cv2.imshow("filter==",filtr)
    cv2.imshow("Result", frame)

    key = cv2.waitKey(25) &0xFF
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()
    





   

count== 3
count== 2
count== 3
count== 3
count== 4
count== 2
count== 1
count== 4
count== 2
count== 4
count== 4
count== 3
count== 1
count== 3
count== 3
count== 3
count== 4
count== 6
count== 3
count== 4
count== 3
count== 4
count== 4
count== 2
count== 4
count== 2
count== 3
count== 3
count== 2
count== 3
count== 3
count== 1
count== 1
count== 0
count== 1
count== 2
count== 2
count== 1
count== 3
count== 0
count== 2
count== 1
count== 0
count== 1
count== 2
count== 0
count== 0
count== 1
count== 1
count== 1
count== 1
count== 4
count== 3
count== 5
count== 2
count== 1
count== 3
count== 5
count== 1
count== 3
count== 2
count== 1
count== 6
count== 5
count== 2
count== 3
count== 1
count== 0
count== 2
count== 4
count== 2
count== 1
count== 5
count== 3
count== 2
count== 0
count== 2
count== 5
count== 0
count== 4
count== 4
count== 3
count== 2
count== 3
count== 4
count== 2
count== 3
count== 2
count== 3
count== 4
count== 2
count== 3
count== 6
count== 1
count== 2
count== 0
count== 1
count== 2
count== 3
count== 4
